In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [2]:
main_link = 'https://eatbydate.com'

In [3]:
def return_category_links(main_link):
    category_links = []
    html_text = requests.get(main_link).text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('div', class_ = "col-cody-12 boxed-cody-white boxed-botbor-green cody-dropshadow padding-zero")
    table_elements = table.find_all('div', class_='col-cody-3')
    for table_element in table_elements:
        link = main_link + table_element.find('a', class_ ='heading-link')['href']
    #     print(link)
        category_links.append(link)

    # Remove the link to contact
    category_links.remove('https://eatbydate.com/contact/')
    return category_links

In [9]:
category_links = return_category_links(main_link)
category_links

['https://eatbydate.com/dairy/',
 'https://eatbydate.com/drinks/',
 'https://eatbydate.com/fruits/',
 'https://eatbydate.com/grains/',
 'https://eatbydate.com/other/',
 'https://eatbydate.com/proteins/',
 'https://eatbydate.com/vegetables/']

In [5]:
# Start with Dairy

def return_sub_category_links(category_link, main_link):
    sub_categories = []
    html_text = requests.get(category_link).text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('div', class_ = "col-cody-12 boxed-cody-white boxed-botbor-green cody-dropshadow padding-zero")
    table_elements = table.find_all('div', class_= lambda c: c and c.startswith('col-cody-4'))
    for table_element in table_elements:
        link = main_link + table_element.find('a', class_ ='heading-link')['href']
        sub_categories.append(link)
    return sub_categories

In [6]:
for category_link in category_links:
    print(category_link)
    sub_categories = return_sub_category_links(category_link, main_link)
    print(sub_categories)

https://eatbydate.com/dairy/
['https://eatbydate.com/drinks/ensure//', 'https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/spreads/butter-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/milk/buttermilk//', 'https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/milk/how-long-does-coffee-mate-last-shelf-life//', 'https://eatbydate.com/dairy/milk/how-long-does-condensed-milk-last-shelf-life//', 'https://eatbydate.com/dairy/cheese/cottage-cheese-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/milk/dairy-coffee-cream-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/cheese/cream-cheese-shelf-life-expiration-date//', 'https://eatbydate.com/how-long-does-egg-nog-last-shelf-life-expiration-date//', 'https://eatbydate.com/eggs-shelf-life-expiration-date//', 'https://eatbydate.com/dairy/milk/how-long-does-evaporated-m

['https://eatbydate.com/proteins/meats/bacon-shelf-life-expiration-date//', 'https://eatbydate.com/proteins/beans-peas/beans-shelf-life-expiration-date//', 'https://eatbydate.com/proteins/meats/beef-shelf-life-expiration-date//', 'https://eatbydate.com/does-beef-jerky-go-bad//', 'https://eatbydate.com/proteins/meats/how-long-does-canned-meat-last-shelf-life-expiration-date//', 'https://eatbydate.com/chia-seeds//', 'https://eatbydate.com/how-long-does-chicken-last-shelf-life-expiration-date//', 'https://eatbydate.com/proteins/poultry/how-long-does-chicken-broth-last-shelf-life//', 'https://eatbydate.com/how-long-do-clams-last//', 'https://eatbydate.com/how-long-does-crab-last//', 'https://eatbydate.com/proteins/meats/deli-meat-shelf-life-expiration-date//', 'https://eatbydate.com/eggs-shelf-life-expiration-date//', 'https://eatbydate.com/fish-shelf-life-expiration-date//', 'https://eatbydate.com/proteins/nuts/flax-seeds//', 'https://eatbydate.com/proteins/meats/ham-shelf-life-expiration

AttributeError: 'NoneType' object has no attribute 'find_all'

Vegetables page is not working from here (page redirection). I will have to use another method to extract them.

In [10]:
# Removing Vegetables from sub_categories
category_links.remove('https://eatbydate.com/vegetables/')

category_links.extend(['https://eatbydate.com/vegetables/fresh-vegetables/'])

In [11]:
category_links

['https://eatbydate.com/dairy/',
 'https://eatbydate.com/drinks/',
 'https://eatbydate.com/fruits/',
 'https://eatbydate.com/grains/',
 'https://eatbydate.com/other/',
 'https://eatbydate.com/proteins/',
 'https://eatbydate.com/vegetables/fresh-vegetables/']

In [12]:
data_list = []

for category_link in category_links:
    sub_categories = return_sub_category_links(category_link, main_link)
    for sub_category in sub_categories:
        print(sub_category)
        if sub_category == 'https://eatbydate.com/turkey-shelf-life-expiration-date//':
            continue
        if sub_category == 'https://eatbydate.com/other/condiments/how-long-does-oil-last//':
            continue 
        html_text = requests.get(sub_category).text
        soup = BeautifulSoup(html_text, 'lxml')
        table = soup.find('div', class_ = "cody-tbl-container")
        if table:
    #         print('yo')
            rows = table.find_all("tr")
    #         print(rows)
            if rows:
                second_heading_index = next((idx for idx, row in enumerate(rows) if row.find("th") and row.find("th").text.strip() == "(Opened)"), None)
                if second_heading_index is not None:
                    headers_1 = [header.get_text(strip=True).lower() for header in rows[0].find_all("th")]
                    for row in rows[1:second_heading_index]:
                        data_dict = {}
                        cells = row.find_all("td")
                        if cells:
                            for idx, cell in enumerate(cells):
                                data_dict[headers_1[idx]] = cell.get_text(strip=True)
                            if any(data_dict.values()):
                                data_list.append(data_dict)
                    headers_2 = [header.get_text(strip=True).lower() for header in rows[second_heading_index].find_all("th")]
                    for row in rows[second_heading_index+1:]:
                        data_dict = {}
                        cells = row.find_all("td")
                        if cells:
                            for idx, cell in enumerate(cells):
                                data_dict[headers_2[idx]] = cell.get_text(strip=True)
                            if any(data_dict.values()):
                                data_list.append(data_dict)
                else:
                    headers = [header.get_text(strip=True).lower() for header in rows[0].find_all("th")]
                    for row in rows[1:]:
                        data_dict = {}
                        cells = row.find_all("td")
                        if cells:
                            for idx, cell in enumerate(cells):
                                data_dict[headers[idx]] = cell.get_text(strip=True)
                            if any(data_dict.values()):
                                data_list.append(data_dict)

        # Add fixed key-value pair for each item
        for entry in data_list:
            new_entry = {"category": category_link, "subcategory":sub_category}
            for key, value in entry.items():
                new_key = 'item_description' if key in ['(unopened)', '(opened)',''] else key
                new_entry[new_key] = value
            if any(key == '(unopened)' for key in entry.keys()):
                new_entry['type'] = 'unopened'
            if any(key == '(opened)' for key in entry.keys()):
                new_entry['type'] = 'opened'
            if any(key == '' for key in entry.keys()):
                new_entry['type'] = 'unspecified'
            entry.clear()
            entry.update(new_entry)

https://eatbydate.com/drinks/ensure//
https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//
https://eatbydate.com/dairy/spreads/butter-shelf-life-expiration-date//
https://eatbydate.com/dairy/milk/buttermilk//
https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//
https://eatbydate.com/dairy/milk/how-long-does-coffee-mate-last-shelf-life//
https://eatbydate.com/dairy/milk/how-long-does-condensed-milk-last-shelf-life//
https://eatbydate.com/dairy/cheese/cottage-cheese-shelf-life-expiration-date//
https://eatbydate.com/dairy/milk/dairy-coffee-cream-shelf-life-expiration-date//
https://eatbydate.com/dairy/cheese/cream-cheese-shelf-life-expiration-date//
https://eatbydate.com/how-long-does-egg-nog-last-shelf-life-expiration-date//
https://eatbydate.com/eggs-shelf-life-expiration-date//
https://eatbydate.com/dairy/milk/how-long-does-evaporated-milk-last-shelf-life//
https://eatbydate.com/hard-boiled-eggs-shelf-

https://eatbydate.com/other/condiments/how-long-does-gravy-last//
https://eatbydate.com/other/snacks/guacamole-shelf-life-expiration-date//
https://eatbydate.com/honey-shelf-expiration-date//
https://eatbydate.com/other/snacks/hummus//
https://eatbydate.com/dairy/ice-cream-shelf-life-expiration-date//
https://eatbydate.com/how-long-does-jam-last-shelf-life-expiration-date//
https://eatbydate.com/other/sweets/how-long-does-jello-last-shelf-life//
https://eatbydate.com/how-long-do-jelly-beans-last-shelf-life-expiration-date//
https://eatbydate.com/ketchup-shelf-life-expiration-date//
https://eatbydate.com/other/leftovers//
https://eatbydate.com/other/sweets/mms//
https://eatbydate.com/how-long-do-marshmallows-last//
https://eatbydate.com/mayonnaise-shelf-life-expiration-date//
https://eatbydate.com/other/baking/how-long-does-molasses-last-shelf-life//
https://eatbydate.com/how-long-does-mustard-last//
https://eatbydate.com/vegetables/fresh-vegetables/how-long-does-nori-last-shelf-life/
h

In [13]:
len(data_list)

893

In [14]:
data_list

[{'category': 'https://eatbydate.com/dairy/',
  'subcategory': 'https://eatbydate.com/drinks/ensure//',
  'item_description': 'Adult Nutrition lastsuntil the',
  '(unopened in pantry)': 'best by date',
  '(opened in fridge)': 'best by date',
  'type': 'unspecified'},
 {'category': 'https://eatbydate.com/dairy/',
  'subcategory': 'https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//',
  'item_description': 'Unrefrigerated Packaging',
  'pantry': 'Past Printed Date',
  'type': 'unopened'},
 {'category': 'https://eatbydate.com/dairy/',
  'subcategory': 'https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//',
  'item_description': 'Soy Milklasts for',
  'pantry': '1 Month',
  'type': 'unopened'},
 {'category': 'https://eatbydate.com/dairy/',
  'subcategory': 'https://eatbydate.com/dairy/milk/soy-rice-almond-milk-substitutes-shelf-life-expiration-date//',
  'item_description': 'Almond Milklasts for',
  'pantr

In [15]:
# Dump the resulting data list to a JSON file
with open('eat_by_date_aryan.json', "w") as json_file:
    json.dump(data_list, json_file, indent=4)